In [ ]:
%%python module Camunda

from js import XMLHttpRequest
from urllib.parse import urlencode
from IPython.display import display, update_display
from robot.api.deco import keyword
from robot.libraries.BuiltIn import BuiltIn

import json
import os

def http(method, path, params=None, data=None):
    request = XMLHttpRequest.new()
    request.open(method, f"{os.environ['CAMUNDA_ENGINE_API']}{path}?{urlencode(params or {})}", False)
    request.setRequestHeader("X-XSRF-TOKEN", os.environ["CAMUNDA_CSRF_TOKEN"])
    request.setRequestHeader("Content-Type", "application/json")
    request.send(json.dumps(data))
    assert request.status in [200, 204], request.responseText
    return json.loads(request.responseText or "null")


class Camunda:
    ROBOT_LIBRARY_SCOPE = 'SUITE'
    ROBOT_LISTENER_API_VERSION = 2

    def __init__(self, key):
        self.ROBOT_LIBRARY_LISTENER = self
        self.definition_key = key
        self.process_ids = []
        self.display_ids = []
        
    def display_instance(self, display_id):
        process = http("GET", f"/history/process-instance/{self.process_ids[-1]}")
        params = dict(processInstanceId=self.process_ids[-1])
        activities = http("GET", f"/history/activity-instance", params)
        incidents = http("GET", f"/incident", params)
        xml = http("GET", f"/process-definition/{process['processDefinitionId']}/xml")
        config = dict(
            style=dict(height="300px"),
            activities=activities,
            incidents=incidents,
        )
        if display_id in self.display_ids:
            update_display({
                "application/bpmn+xml": xml["bpmn20Xml"],
                "application/bpmn+json": json.dumps(config),
            }, raw=True, display_id=display_id)
        else:
            display({
                "application/bpmn+xml": xml["bpmn20Xml"],
                "application/bpmn+json": json.dumps(config),
            }, raw=True, display_id=display_id)
        self.display_ids.append(display_id)

    
    def get_instances(self):
        params = dict(processDefinitionKey=self.definition_key)
        return http("GET", "/process-instance", params)
    
    def get_activities(self):
        params = dict(processInstanceId=self.process_ids[-1])
        return http("GET", f"/history/activity-instance", params)
    
    def get_external_tasks(self):
        assert self.process_ids, "No process managed by Camunda library instance is running"
        params = dict(processInstanceId=self.process_ids[-1])
        return http("GET", "/external-task", params)
    
    @keyword(tags=["bpmn"])
    def complete_external_task(self, topic, variables):
        worker_id = f"{self.__class__.__name__}:{id(self)}"
        data = dict(
            workerId=worker_id,
            maxTasks=1,
            topics=[dict(
                topicName=topic,
                lockDuration=1000
            )]
        )
        external_tasks = http("POST", "/external-task/fetchAndLock", data=data)
        external_tasks = [x for x in external_tasks if x["processInstanceId"] == self.process_ids[-1]]
        assert external_tasks, "No external tasks available"
        for external_task in external_tasks:
            data = dict(workerId=worker_id, variables=variables)
            http("POST", f"/external-task/{external_task['id']}/complete", data=data)
    
    @keyword(tags=["bpmn"])
    def start_instance(self):
        data = dict(variables={})
        instance = http("POST", f"/process-definition/key/{self.definition_key}/start", data=data)
        self.process_ids.append(instance["id"])
    
    @keyword(tags=["bpmn"])
    def stop_instance(self):
        if self.process_ids:
            process_id = self.process_ids.pop()
            process = http("GET", f"/history/process-instance/{process_id}")
            if process["state"] == "ACTIVE":
                params = dict(
                    skipCustomListeners="true",
                    skipIoMappings="true",
                    failIfNotExist="false",
                )
                http("DELETE", f"/process-instance/{process_id}", params)
            
    def end_keyword(self, name, attrs):
        if "bpmn" in attrs['tags'] and self.process_ids :
            BuiltIn().sleep("1 s")
            self.display_instance(f"{id(self)}")


In [ ]:
*** Settings ***

Library  Camunda  key=traffic-data-loader
Library  Collections

Suite setup  Start instance
Suite teardown  Stop instance

In [ ]:
*** Keywords ***

Number of instances should be
    [Arguments]  ${number}  ${error}
    ${instances}=  Get instances
    ${number of instances}=  Get length  ${instances}
    Should be equal as integers  ${number of instances}  ${number}  ${error}

Should contain external task
    [Arguments]  ${topic}  ${error}
    ${external tasks}=  Get external tasks
    ${counter}=  Set variable  ${{0}}
    FOR  ${external task}  IN  @{external tasks}
        IF  ${{"${external task}[topicName]" == "${topic}"}}
            ${counter}=  Set variable  ${{${counter} + 1}}
        END
    END
    Should be true  ${{${counter} > 0}}  ${error}

Should not contain external task
    [Arguments]  ${topic}  ${error}
    ${external tasks}=  Get external tasks
    FOR  ${external task}  IN  @{external tasks}
        Should not be equal  ${external task}[topicName]  ${topic}  ${error}
    END

In [ ]:
*** Test Cases ***

Test process is properly setup and teardown
    Number of instances should be  1  Process instance should be running
    
Test start and stop instance
    Start instance
    Number of instances should be  2  A new process instance should be running
    [Teardown]  Stop instance

In [ ]:
*** Test Cases ***

Producer task is the first task
    Should contain external task
    ...  Produce traffic data work items
    ...  Process should start with Producer task
    Should not contain external task
    ...  Consume traffic data work item
    ...  Process should start with Producer tas

In [ ]:
*** Keywords ***

Produce traffic data work items
    [Arguments]  ${count}=1
    Should contain external task
    ...  Produce traffic data work items
    ...  Process should start with Producer task
    ${item}=  Create dictionary
    ${data}=  Create List
    Repeat keyword  ${count} times
    ...  Append to list  ${data}  ${item}  
    ${variable}  Create dictionary  value=${{json.dumps(${data})}}  type=Json
    ${variables}=  Create dictionary  traffic_data=${variable}
    Complete external task
    ...  Produce traffic data work items
    ...  ${variables}
    Should not contain external task
    ...  Produce traffic data work items
    ...  Process should continue without Producer task
    [Return]  ${data}

Consume traffic data work item
    Should contain external task
    ...  Consume traffic data work item
    ...  Process should start with Consumer task
    ${item}=  Create dictionary
    ${variable}  Create dictionary  value=OK  type=String
    ${variables}=  Create dictionary  status=${variable}
    Complete external task
    ...  Consume traffic data work item
    ...  ${variables}

In [ ]:
*** Test Cases ***

Happy Path
    Number of instances should be  1  Process should have been started
    Produce traffic data work items  5
    Repeat keyword  5  Consume traffic data work item    
    Should not contain external task
    ...  Consume traffic data work item
    ...  Process should have been completed
    Number of instances should be  0  Process should have been completed

In [ ]:
*** Keywords ***

Consume traffic data work item with Service failure
    Should contain external task
    ...  Consume traffic data work item
    ...  Process should continue with Consumer task
    ${item}=  Create dictionary
    ${variable}  Create dictionary  value=API_ERROR  type=String
    ${variables}=  Create dictionary  status=${variable}
    Complete external task
    ...  Consume traffic data work item
    ...  ${variables}
    Should contain completed activity
    ...  service_failure
    ...  Service failure BPMN error should have been triggered

Consume traffic data work item with Validation failure
    Should contain external task
    ...  Consume traffic data work item
    ...  Process should continue with Consumer task
    ${item}=  Create dictionary
    ${variable}  Create dictionary  value=INVALID_DATA  type=String
    ${variables}=  Create dictionary  status=${variable}
    Complete external task
    ...  Consume traffic data work item
    ...  ${variables}
    Should contain completed activity
    ...  validation_failure
    ...  Validation failure BPMN error should have been triggered
    
Should contain completed activity
    [Arguments]  ${activity id}  ${error}
    ${activities}=  Get activities
    ${counter}=  Set variable  ${{0}}
    FOR  ${activity}  IN  @{activities}
        IF  ${{"${activity}[activityId]" == "${activity id}"}}
            IF  ${{"${activity}[endTime]" != "None"}}
                ${counter}=  Set variable  ${{${counter} + 1}}
            END
        END
    END
    Should be true  ${{${counter} > 0}}  ${error}

Should not contain completed activity
    [Arguments]  ${activity id}  ${error}
    ${activities}=  Get activities
    FOR  ${activity}  IN  @{activities}
        IF  ${{"${activity}[activityId]" == "${activity id}"}}
            IF  ${{"${activity}[endTime]" != "None"}}
                Fail  ${error}
            END
        END
    END

In [ ]:
*** Test Cases ***

Fail process with service failure
    Number of instances should be  1  Process should have been started
    Produce traffic data work items  1
    Consume traffic data work item with Service failure
    Should not contain completed activity
    ...  validation_failure
    ...  Validation failure should not have been triggered
    Number of instances should be  1  Process should not have been completed
    Should contain external task
    ...  Consume traffic data work item
    ...  Process should continue with Consumer task

In [ ]:
*** Test Cases ***

Fail process with validation failure
    Number of instances should be  1  Process should have been started
    Produce traffic data work items  1
    Consume traffic data work item with Validation failure
    Should not contain completed activity
    ...  service_failure
    ...  Service failure should not have been triggered
    Number of instances should be  0  Process should have been completed

In [ ]:
*** Test Cases ***

Full Coverage
    Number of instances should be  1  Process should have been started
    Produce traffic data work items  2
    Consume traffic data work item
    Consume traffic data work item with service failure
    Consume traffic data work item with validation failure
    Number of instances should be  0  Process should have been completed